In [1]:
import os
import json
from openai import OpenAI
from dotenv import load_dotenv
from pprint import pprint

In [2]:
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
if openai_api_key:
    print(f"OpenAI API Key exists and begins:{openai_api_key[:12]}")
else:
    print("OpenAI API Key not set")

OpenAI API Key exists and begins:sk-proj-TVye


In [3]:
client = OpenAI()

In [4]:
MODEL = "gpt-4o-mini"

In [5]:
print(f"MODEL: {MODEL}")

MODEL: gpt-4o-mini


In [6]:
source_data = """
Peter is a vegan and won't eat any meat. He has been a vegan for over five years when he met his current wife Angela. They have two dogs, Roxy and Petra, and they both eat meat.

They both work in London but live in Brighton near Seven Dials.

They travel a lot and have visited the following countries in the last year - Italy, France and Germany.

They have an active YouTube channel where they post videos about their travels.

They have a cat named Marmalade.

Brighton is based on the South Coast of England and is known for its beaches and nightlife.

Brighton is about 50 miles south of London.
"""

In [7]:
system_message = """You are a teachability agent. You examine a conversation listed between <conv> and </conv> and output a list of pertinent facts as well as a concise summary. The OUTPUT FORMAT *must have* in the following JSON FORMAT:

{
    "summary": <SUMMARY>,
    "number_of_facts": <NUMBER_OF_FACTS>,
    "facts": [<FACTS>]
}

A fact is a dictionary with the following keys: "fact" and "catgerory".

Here are some examples of facts:

{"fact": "Charles is a vegan and won't eat any meat.", "category": "personal"}
{"fact": "Charles works in Brighton", "category": "work"}
{"fact": "They have four dogs", "category": "pets"}

## NOT A FACT
This is not a fact as it does not refer to a person:
"London is a city and the capital of England"

## NUMBER_OF_FACTS
<NUMBER_OF_FACTS> stores the number of facts in the "facts" list
*Be as specific as you can about the categories*
"""

In [8]:
data = """Peter is a vegan and won't eat any meat. He has been a vegan for over five years when he met his current wife Angela. They have two dogs, Roxy and Petra, and they both eat meat.

They both work in London but live in Brighton near Seven Dials.

They travel a lot and have visited the following countries in the last year - Italy, France and Germany.

They have an active YouTube channel where they post videos about their travels.

They have a cat named Marmalade.

Brighton is based on the South Coast of England and is known for its beaches and nightlife."""

In [9]:
system_message += "<conv>" + data + "</conv>"

In [10]:
prompts = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": ""},
]

In [11]:
response = client.chat.completions.create(model=MODEL, messages=prompts, temperature=0)
res = response.choices[0].message.content.replace("\n", "")

In [12]:
output = json.loads(res)

Having extracted information, we can store this in 'short term memory' - app level cache - or long term menory - stored in a DB for example and retrieved as needed and added to the SYSTEM MESSAGE.

In [13]:
pprint(output)

{'facts': [{'category': 'personal',
            'fact': "Peter is a vegan and won't eat any meat."},
           {'category': 'personal',
            'fact': 'Peter has been a vegan for over five years.'},
           {'category': 'personal', 'fact': 'Peter is married to Angela.'},
           {'category': 'pets',
            'fact': 'They have two dogs named Roxy and Petra.'},
           {'category': 'pets', 'fact': 'They have a cat named Marmalade.'},
           {'category': 'work', 'fact': 'They both work in London.'},
           {'category': 'location',
            'fact': 'They live in Brighton near Seven Dials.'},
           {'category': 'travel',
            'fact': 'They have visited Italy, France, and Germany in the last '
                    'year.'}],
 'number_of_facts': 8,
 'summary': 'Peter is a vegan who has been so for over five years. He lives in '
            'Brighton with his wife Angela, and they have two dogs and a cat. '
            'They both work in London and trav

In [14]:
pprint(output["summary"])
pprint(output["facts"])

('Peter is a vegan who has been so for over five years. He lives in Brighton '
 'with his wife Angela, and they have two dogs and a cat. They both work in '
 'London and travel frequently, having visited Italy, France, and Germany in '
 'the past year. They also run an active YouTube channel focused on their '
 'travels.')
[{'category': 'personal', 'fact': "Peter is a vegan and won't eat any meat."},
 {'category': 'personal',
  'fact': 'Peter has been a vegan for over five years.'},
 {'category': 'personal', 'fact': 'Peter is married to Angela.'},
 {'category': 'pets', 'fact': 'They have two dogs named Roxy and Petra.'},
 {'category': 'pets', 'fact': 'They have a cat named Marmalade.'},
 {'category': 'work', 'fact': 'They both work in London.'},
 {'category': 'location', 'fact': 'They live in Brighton near Seven Dials.'},
 {'category': 'travel',
  'fact': 'They have visited Italy, France, and Germany in the last year.'}]
